In [ ]:
import torch
data_cora = torch.load('cora_fixed.pt')

In [ ]:
#把节点的文本属性转换为向量
from sentence_transformers import SentenceTransformer
model_ST = SentenceTransformer('paraphrase-MiniLM-L6-v2')
sentences = data_cora.raw_text
sentence_embeddings = model_ST.encode(sentences)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
# 创建KMeans实例，设置簇的数量
kmeans = KMeans(n_clusters=7, random_state=0).fit(sentence_embeddings)#random_state参数是一个种子值，用于初始化随机数生成器
# 获取每个点对应的簇标签
labels = kmeans.labels_

# 获取质心
centroids = kmeans.cluster_centers_

# 计算每个点到它对应的簇心的距离
distances = np.sqrt(np.sum((sentence_embeddings - centroids[labels]) ** 2, axis=1))#欧式距离

# 使用 argsort 对距离进行升序排序，并获取排序后的索引
sorted_indices = np.argsort(distances)

# 使用排序后的索引来获取对应的点
sorted_points = sentence_embeddings[sorted_indices]

print("按距离升序排序的点的索引:", sorted_indices)

In [ ]:
def generate_sublist_by_indices(original_list, indices):
    # 使用列表推导来生成新的列表
    return [original_list[i] for i in indices if i < len(original_list)]
def fun(num,responses):
    long_index = []
    for i in range(len(responses)):
        if (len(responses[i])>num):
            long_index.append(i)
    return long_index

# 调用函数并打印结果
sorted_data_cora_text = generate_sublist_by_indices(data_cora.raw_text, sorted_indices)

In [ ]:
#generate itial annotation
import time
import openai
client = OpenAI(
    api_key = "sk-ylwjcmyaipuproleazzfgpmatrduhgtqeemcajcegvodlwde",
    base_url = "https://api.siliconflow.cn/v1",
)
# 您的文本列表
text_list = sorted_data_cora_text

# 用于存储Kimi的回答
responses = []
# 遍历文本列表，发送请求并获取回答
for text in text_list:
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V2.5",
        messages=[
            {"role": "system", "content": """You are an artificial intelligence assistant.You specialize in annatation for classification tasks .
        I will give you a paper, and you need to give me the category of this paper.
        There are following categories: [Rule_Learning,Neural_Networks,Case_Based,Genetic_Algorithms,Theory,Reinforcement_Learning,Probabilistic_Methods]
        Provide your 3 best guesses and a confidence number that each is correct (0 to 100) for the following question from most probable to least. The sum of one work's confidence should be 100.
        An output example: Genetic_Algorithms: 50,Probabilistic_Methods: 30,Rule_Learning: 20"""},
            {"role": "user", "content": text},
        ],
        temperature=0.5,
    )
    time.sleep(1)
    # 将回答添加到responses列表中
    responses.append(completion.choices[0].message.content)
import pickle
# 打开一个文件用于二进制写入，并使用 pickle 保存列表
with open('list.pkl', 'wb') as file:
    pickle.dump(responses, file)

In [ ]:
# 最新的correction prompt
import openai
from openai import OpenAI

client = OpenAI(
    api_key = "sk-ylwjcmyaipuproleazzfgpmatrduhgtqeemcajcegvodlwde",
    base_url = "https://api.siliconflow.cn/v1",
)
index_responses_wrong = fun(100,responses)
text_list = generate_sublist_by_indices(responses, index_responses_wrong)
responses_wrong_2 = []
for text in text_list:
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V2.5",
        messages=[
            {"role": "system", "content": """You are an artificial intelligence assistant.You specialize in annatation for classification tasks .
        I will give you a text consisting of wrong format. You need to correct it.
        An right output example: Genetic_Algorithms: 50,Probabilistic_Methods: 30,Rule_Learning: 20
        An wrong output example: Probabilistic_Methods: 60, Bayesian models...,Theory: 30, The paper provides historical review...,Neural_Networks: 10, While the paper discusses..."""},
            {"role": "user", "content": text},
        ],
        temperature=0.9,
    )
    #time.sleep(1)
    # 将回答添加到responses列表中
    responses_wrong_2.append(completion.choices[0].message.content)

In [ ]:
def merge_wrong2right(wrong,right,index_wrong):
    i = index_wrong[0]
    j = 0
    for ele in wrong:
        right[i] = ele
        j = j + 1
        if (j>=len(wrong)):
            return right
        else:
            i = index_wrong[j]
    return right
responses_right = merge_wrong2right(responses_wrong_2, responses, index_responses_wrong)

In [ ]:
# prompt for finding maximum scores 
import openai
from openai import OpenAI
import time
client = OpenAI(
    api_key = "sk-ylwjcmyaipuproleazzfgpmatrduhgtqeemcajcegvodlwde",
    base_url = "https://api.siliconflow.cn/v1",
)
text_list = responses_right[0:2000]
result2000 = []
for text in text_list:
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V2.5",
        messages=[
            {"role": "system", "content": """You are an artificial intelligence assistant.You specialize in annatation for classification tasks .
        I will give you a text as input. You need to find category with maximum number as output.
        For example: 
        input:Genetic_Algorithms: 50,Probabilistic_Methods: 30,Rule_Learning: 20
        output: Genetic_Algorithms"""},
            {"role": "user", "content": text},
        ],
        temperature=0.9,
    )
    time.sleep(0.5)
    # 将回答添加到responses列表中
    result2000.append(completion.choices[0].message.content)
import pickle
# 打开一个文件用于二进制写入，并使用 pickle 保存列表
with open('result2000.pkl', 'wb') as file:
    pickle.dump(result2000, file)

In [ ]:
def compare(index,left,right):
    tag = []
    j = 0
    for i in index:
        if left[i] == right[j]:
            tag.append(True)
        else:
            tag.append(False)
        if (j>=(len(right)-1)):
            return tag
        else:
            j = j + 1
    return tag

tag =compare(sorted_indices,data_cora.category_names,result2000)
chunked_tag = [tag[i:i+200] for i in range(0, len(tag), 200)]

def accuracy(tag):
    sum = 0
    for i in tag:
        if i == True:
            sum = sum + 1
    acc = sum / len(tag)
    return acc

acc_vec = []
for sublist in chunked_tag:
    acc_vec.append(accuracy(sublist))

In [ ]:
# draw bar graph
import matplotlib.pyplot as plt

# vecor
vector = acc_vec

# Plotting the bar chart with blue color
plt.bar(range(len(vector)), vector, color='blue')

# Adding title and labels
plt.title('CORA')
plt.xlabel('Region Index')
plt.ylabel('Average accuracy')

# Setting the range of the axes
plt.xlim(-1, len(vector))  
plt.xticks(range(0, len(vector)))
plt.ylim(0.55, max(vector) + 0.05)  

for i, value in enumerate(vector):
    plt.text(i , value + 0.01, str(value), ha='center')  # 注意这里的i+1
# Display the plot
plt.show()